In [1]:
import lib_non_local_games as nlg
import numpy as np
import cvxpy as cp
import functools as fc
from operator import mul

In [2]:
cp.installed_solvers()

['ECOS', 'ECOS_BB', 'MOSEK', 'OSQP', 'SCS']

Parameter of the CHSH game

In [3]:
dimA1 = 2
dimA2 = 2
dimQ1 = 2
dimQ2 = 2

dimT = 2
dimS = 2

probQ1 = (.5,.5)
probQ2 = (.5,.5)

# CHSH game ($n_1 = 1$, $n_2 = 1$) with NPA constraints

In [4]:
# Subsystems A1 Q1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)
dim_A1Q1A2Q2 = fc.reduce(mul, subs_A1Q1A2Q2, 1)

# Subsystems A2 Q1 Q2
subs_A2Q1Q2 = (dimA2,dimQ1,dimQ2)
indices_A2Q1Q2 = nlg.indices_list(subs_A2Q1Q2)

# Subsystems A1 Q1 Q2
subs_A1Q1Q2 = (dimA1,dimQ1,dimQ2)
indices_A1Q1Q2 = nlg.indices_list(subs_A1Q1Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)
dim_A1Q1 = fc.reduce(mul, subs_A1Q1, 1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)
dim_A2Q2 = fc.reduce(mul, subs_A2Q2, 1)

# Subsystems A1 A2
subs_A1A2 = (dimA1,dimA2)
indices_A1A2 = nlg.indices_list(subs_A1A2)

# Subsystems Q1 Q2
subs_Q1Q2 = (dimQ1,dimQ2)
indices_Q1Q2 = nlg.indices_list(subs_Q1Q2)

# Subsystems T hat(T) S hat(S)
subs_TTSS = (dimT,dimT,dimS,dimS)
indices_TTSS = nlg.indices_list(subs_TTSS)
dim_TTSS = fc.reduce(mul, subs_TTSS, 1)

# Additional dimensions
dim_TSS = dimT * dimS**2
dim_TT = dimT**2
dim_SS = dimS**2

# State on subsystem T
rhoT = np.identity(dimT)/dimT

In [5]:
BtI = lambda seq : nlg.seqtoint(seq, subs_A1Q1A2Q2)

In [6]:
# Maximally entangled vectors between T|S and TT|SS
phi_TS = nlg.bipartite_unnorm_max_entangled_state(dimT)
phi_TSTS = nlg.tensor([phi_TS,phi_TS])

# Maximally mixed states between TT|SS (correct order of subsystems)
Phi_TSTS = np.outer(phi_TSTS,phi_TSTS)
P = nlg.permutation_matrix((0,1,2,3), (0,2,1,3), (dimT, dimT,dimT, dimT))
Phi_TTSS = P @ Phi_TSTS @ P

The program

In [7]:
## VARIABLES

# The (sub-normalized) states we optimize over
rho_TTSS = []
for i in map(BtI,indices_A1Q1A2Q2):
    rho_TTSS.append( cp.Variable((dim_TTSS,dim_TTSS),hermitian=True) )

## OBJECTIVE FUNCTION

# The swap operator is
F_TTSS = nlg.permutation_matrix((0,1), (1,0), (dim_TT, dim_SS))

# The rule function is
rule_A1Q1A2Q2 = nlg.CHSH_rule_function_A1Q1A2Q2
#rule_A1Q1A2Q2 = random_rule_function_A1Q1A2Q2

# The object function is
obj_fun_components = [rule_A1Q1A2Q2(*index) * cp.trace( cp.matmul(Phi_TTSS,rho_TTSS[BtI(index)]) )
                      for index in indices_A1Q1A2Q2]

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []
    
# 2) rho_TTSS are (sub-normalized) quantum states
# 2a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_TTSS[i]) for i in map(BtI,indices_A1Q1A2Q2)]) - 1 == 0 )

# 2b) positive semidefinite matrices
for i in map(BtI,indices_A1Q1A2Q2):
    constraints.append( rho_TTSS[i] >> 0 )
    
# 3) First linear constraint
for a2,q1,q2 in indices_A2Q1Q2:
    indices_A1q1a2q2 = [BtI([a,q1,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a2q2 = [BtI([a,q,a2,q2]) for a,q in indices_A1Q1]
    
    lhs = sum([rho_TTSS[i] for i in indices_A1q1a2q2])
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_A1Q1a2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_TSS])
    rhs = cp.Constant(probQ1[q1]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 4) Second linear constraint
P = nlg.permutation_matrix((0,1,2,3),(1,0,2,3),subs_TTSS)

for a1,q1,q2 in indices_A1Q1Q2:
    indices_a1q1A2q2 = [BtI([a1,q1,a,q2]) for a in range(dimA2)]
    indices_a1q1A2Q2 = [BtI([a1,q1,a,q]) for a,q in indices_A2Q2]
    
    lhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2q2])
    lhs = cp.matmul( cp.matmul(P,lhs_variable) , P )
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2Q2])
    rhs_permuted = cp.matmul( cp.matmul(P,rhs_variable) , P )
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_TSS])
    rhs = cp.Constant(probQ2[q2]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )

# 5) NPA style constraint (see PhysRevLett.98.010401)

# Introduce normalization factor
renorm = lambda x,y : dimT**2/(probQ1[x]*probQ2[y])

# The P matrix containing the information about the variables rho_TTSS is give by
P = []

for a1,q1 in indices_A1Q1:
    P_row = [renorm(q1,q2)*cp.trace(Phi_TTSS @ rho_TTSS[BtI([a1,q1,a2,q2])]) for a2,q2 in indices_A2Q2]
    P.append(P_row)

P = cp.bmat(P)

# The Q matrix containing information on the variables rho_TTSS and new variables as well
Q = []

for a1,q1 in indices_A1Q1:
    Q_row = []
    for a1p,q1p in indices_A1Q1:
        if q1 == q1p:
            if a1 == a1p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[BtI([a1,q1,a2,q2])]) for a2,q2 in indices_A2Q2])/dimQ2
            else:
                val = cp.Constant(0)
        else:
            val = cp.Variable()
        Q_row.append(val)
    Q.append(Q_row)

Q = cp.bmat(Q)

# The R matrix containing information on the variables rho_TTSS and new variables as well
R = []

for a2,q2 in indices_A2Q2:
    R_row = []
    for a2p,q2p in indices_A2Q2:
        if q2 == q2p:
            if a2 == a2p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[BtI([a1,q1,a2,q2])]) for a1,q1 in indices_A1Q1])/dimQ1
            else:
                val = cp.Constant(0)
        else:
            val = cp.Variable()
        R_row.append(val)
    R.append(R_row)

R = cp.bmat(R)

# Constructing vector v of dimension dimA1*dimQ1+dimA2*dimQ2
v = []

for a1,q1 in indices_A1Q1:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTSS @ rho_TTSS[BtI([a1,q1,a2,q2])]) for a2,q2 in indices_A2Q2])/dimQ2)
    
for a2,q2 in indices_A2Q2:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTSS @ rho_TTSS[BtI([a1,q1,a2,q2])]) for a1,q1 in indices_A1Q1])/dimQ1)
    
v = cp.bmat([v])
w = cp.vstack([cp.Constant([[1]]),v.T])

# Builiding the matrix M that should be positive semi-definite (NPA constraint)
M = cp.vstack([cp.hstack([Q,P]),cp.hstack([P.T,R])])
M = cp.vstack([v,M])
M = cp.hstack([w,M])
      
constraints.append( M >> 0 )
constraints.append( M - M.T == 0 )

# 6) PPT criterium
for i in map(BtI,indices_A1Q1A2Q2):
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,1,0,0)) >> 0 )
    
# Write the problem
prob = cp.Problem(cp.Maximize(cp.real(object_function)), constraints)

Solve the problem

In [8]:
prob.solve(verbose=True,solver='MOSEK')



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 74216           
  Cones                  : 0               
  Scalar variables       : 6288            
  Matrix variables       : 65              
  Integer variables      : 0               

Optimizer started.
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 74216           
  Cones                  : 0               
  Scalar variables       : 6288            
  Matrix variables       : 65              
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            

0.8535533905968526

A different constraint, used in Application I of the PRL paper

In [33]:
# The P matrix containing the information about the variables rho_TTSS is give by
P = []

for q1 in range(dimQ1):
    P_row = []
    for q2 in range(dimQ2):
        # Normalization constant to get P(a1,a2|q1,q2)
        norm = cp.Constant( dimT**2/(probQ1[q1]*probQ2[q2]) )
        # Compute the correlation function C_{q1,q2} = sum_{a1,a2} (-1)^{a1*a2} p(a1,a2|q1,q2)
        C = sum([(-1)**(a1+a2)*norm*cp.trace(cp.matmul(Phi_TTSS,rho_TTSS[BtI([a1,q1,a2,q2])])) for a1,a2 in indices_A1A2])
        P_row.append(C)
    P.append(P_row)

P = cp.bmat(P)

# Introduce new variables Q and R, symmetic matrices whose elements are < 1

# Q is dimQ1 x dimQ1
Q = cp.Variable((dimQ1,dimQ1),symmetric=True)
# R is dimQ2 x dimQ2
R = cp.Variable((dimQ2,dimQ2),symmetric=True)

# 5.1) The matrix M = [[Q,P],[P^T,R]] is PSD (NPA constraint)
M = cp.bmat([[Q,P],[P.T,R]])
constraints.append( M >> 0 )

# 5.2) Assumptions on projective nature of measurement on A and B
# Assumption over Q
for q in range(dimQ1):
    constraints.append( Q[q,q] - 1 == 0 )
    
# Assumption over R
for q in range(dimQ2):
    constraints.append( R[q,q] - 1 == 0 )

# I$_{3322}$ ($n_1=1, n_2=1 $) with NPA constraints 

In [3]:
# I3322 rule matrix
def I3322_rule_function(a1,q1,a2,q2,eta=1.):
    
    index = str(a1)+str(a2)+str(q1)+str(q2)

    I3322_rules = {"0000":1./3-1./(3*eta), 
                   "0001":2./3-1./(3*eta), 
                   "0002":1.-1./(3*eta), 
                   "0010":1./3,
                   "0011":2./3,
                   "0012":-1.,
                   "0020":1./3,
                   "0021":-4./3,
                   "0100":-1./(3*eta), 
                   "0101":-1./(3*eta), 
                   "0102":-1./(3*eta), 
                   "1000":-2./3,
                   "1001":-1./3,
                   "1010":-2./3,
                   "1011":-1./3,
                   "1020":-2./3,
                   "1021":-1./3}
    
    try:
        out = I3322_rules[index]
    except KeyError:
        out = 0
    
    return out

def seqtoint(sequence,dimension):
    for n,d in enumerate(dimension[1:]):
        sequence =[d*element for element in sequence[:n+1]] + list(sequence[n+1:])
    return sum(sequence)

In [4]:
# Parameter of the I3322 game
dimA1 = 2
dimA2 = 2
dimQ1 = 3
dimQ2 = 3

dimT = 2
dimS = 2

probQ1 = (1/3,1/3,1/3)
probQ2 = (1/3,1/3,1/3)

eta = 1

In [5]:
# Subsystems A1 Q1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)

# Subsystems Q1 A2 Q2
subs_Q1A2Q2 = (dimQ1,dimA2,dimQ2)
indices_Q1A2Q2 = nlg.indices_list(subs_Q1A2Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)

# Subsystems  A1 Q1 Q2
subs_A1Q1Q2 = (dimA1,dimQ1,dimQ2)
indices_A1Q1Q2 = nlg.indices_list(subs_A1Q1Q2)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)

# Subsystems T1 T2
subs_TT = (dimT,dimT)
dim_TT = fc.reduce(mul, subs_TT, 1)

# Subsystems T1 T2 S1 S2
subs_TTSS = (dimT,dimT,dimS,dimS)
dim_TTSS = fc.reduce(mul, subs_TTSS, 1)

# Dimensions
dim_TSS = dimT * dimS**2
dim_SS = dimS**2
dim_A1Q1 = dimA1*dimQ1
dim_A2Q2 = dimA2*dimQ2

# State on subsystem T
rhoT = np.identity(dimT)/dimT

# Here we define the function from sequence to integer for the I3322
StI = lambda seq : seqtoint(seq, subs_A1Q1A2Q2)

In [6]:
# Maximally entangled vectors between T|S and TT|SS
phi_TS = nlg.bipartite_unnorm_max_entangled_state(dimT)
phi_TSTS = nlg.tensor([phi_TS,phi_TS])

# Maximally mixed states between TT|SS (correct order of subsystems)
Phi_TSTS = np.outer(phi_TSTS,phi_TSTS)
P = nlg.permutation_matrix((0,1,2,3), (0,2,1,3), (dimT, dimT,dimT, dimT))
Phi_TTSS = P @ Phi_TSTS @ P

In [7]:
## VARIABLES 

# The (sub-normalized) states we optimize over
rho_TTSS = []
shape_TTSS = (dim_TTSS, dim_TTSS)

for index in indices_A1Q1A2Q2:
    rho_TTSS.append( cp.Variable(shape_TTSS,symmetric=True) )

## OBJECTIVE FUNCTION

F_TTSS = nlg.permutation_matrix((0,1), (1,0), (dim_TT, dim_SS))

# The object function is
obj_fun_components = []

for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    v_a1q1a2q2 = I3322_rule_function(a1,q1,a2,q2,eta)
    obj_fun_components.append( v_a1q1a2q2 * cp.trace( cp.matmul(F_TTSS,rho_TTSS[StI([a1,q1,a2,q2])]) ) )
    
object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []

# 1) rho_TTSS are (sub-normalized) quantum states
# 1a) trace of the sum is 1
trace_rho = sum([cp.trace(rho_TTSS[StI(index)]) for index in indices_A1Q1A2Q2])
constraints.append( trace_rho - 1 == 0 )

# 1b) positive semidefinite matrices
for index in indices_A1Q1A2Q2:
    constraints.append( rho_TTSS[StI(index)] >> 0 )

    
# 2) First linear constraint
for q1,a2,q2 in indices_Q1A2Q2:
    indices_A1q1a2q2 = [StI([a,q1,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a2q2 = [StI([a,q,a2,q2]) for a,q in indices_A1Q1]

    lhs = sum([rho_TTSS[i] for i in indices_A1q1a2q2])

    rhs_variable = sum([rho_TTSS[i] for i in indices_A1Q1a2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_TSS])
    rhs = probQ1[q1] * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )

    
# 3) Second linear constraint

# The permutation matrix swaps T1 and T2
P = cp.Constant(nlg.permutation_matrix((0,1,2,3), (1,0,2,3), subs_TTSS))

for a1,q1,q2 in indices_A1Q1Q2:
    indices_a1q1A2q2 = [StI([a1,q1,a,q2]) for a in range(dimA2)]
    indices_a1q1A2Q2 = [StI([a1,q1,a,q]) for a,q in indices_A2Q2]

    lhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2q2])
    lhs = cp.matmul(cp.matmul(P,lhs_variable),P)

    rhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2Q2])
    rhs_permuted = cp.matmul(cp.matmul(P,rhs_variable),P)
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_TSS])
    rhs = probQ2[q2] * cp.kron(rhoT, rhs_partial)

    constraints.append( lhs - rhs == 0 )

    
# 4) PPT criterium
for index in indices_A1Q1A2Q2:
    constraints.append( nlg.partial_transpose(rho_TTSS[StI(index)],subs_TTSS,(0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[StI(index)],subs_TTSS,(1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[StI(index)],subs_TTSS,(0,1,0,0)) >> 0 )


# 5) NPA constraints
P = []

for a1,q1 in indices_A1Q1:
    P_row = []
    for a2,q2 in indices_A2Q2:
        # Normalization constant to get p(a1,a2|q1,q2)
        norm = cp.Constant( dimT**2/(probQ1[q1]*probQ2[q2]) )
        # Compute the p(a1,a2|q1,q2)
        value = norm*cp.trace(cp.matmul(F_TTSS,rho_TTSS[StI([a1,q1,a2,q2])]))
        P_row.append(value)
    P.append(P_row)

P = cp.bmat(P)

# Introduce new variables Q and R, symmetic matrices whose elements are < 1

Q = []
for a1,q1 in indices_A1Q1:
    Q_row = []
    for a1p,q1p in indices_A1Q1:
        norm = cp.Constant( dimT**2/(probQ1[q1]*probQ1[q1p]))
        if q1 == q1p:
            if a1 == a1p:
                val = norm*sum([cp.trace(cp.matmul(F_TTSS,rho_TTSS[StI([a1,q1,a,q1p])])) for a in range(dimA1)])
            else:
                val = cp.Constant(0)
        else:
            val = cp.Variable()
        Q_row.append(val)
    Q.append(Q_row)

R = []
for a2,q2 in indices_A2Q2:
    R_row = []
    for a2p,q2p in indices_A2Q2:
        norm = cp.Constant( dimT**2/(probQ2[q2]*probQ2[q2p]))
        if q2 == q2p:
            if a2 == a2p:
                val = norm*sum([cp.trace(cp.matmul(F_TTSS,rho_TTSS[StI([a,q2p,a2,q2])])) for a in range(dimA2)])
            else:
                val = cp.Constant(0)
        else:
            val = cp.Variable()
        R_row.append(val)
    R.append(R_row)
    
Q = cp.bmat(Q)
R = cp.bmat(R)
    
# 5.2) The matrix M = [[Q,P],[P^T,R]] is PSD (NPA constraint)
M = cp.bmat([[Q,P],[P.T,R]])
constraints.append( M >> 0 )
constraints.append( M - M.T == 0 )

## PROBLEM

# Write the problem
prob = cp.Problem(cp.Maximize(object_function), constraints)

In [8]:
prob.solve(verbose=True,solver='MOSEK')



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 46369           
  Cones                  : 0               
  Scalar variables       : 4944            
  Matrix variables       : 145             
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 46369           
  Cones                  : 0               
  Scalar variables       : 4944            
  Matrix variables       : 145             
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 38439
Optimizer  - Cones                  : 1
Optimizer  - Scalar variabl

0.04066949169746176

In [9]:
print(0.04066949169746176*9)

0.3660254252771558


In the NPA paper, they've got 0.3333 for $I_{3322}$ for the first level